In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')
%matplotlib inline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from itertools import combinations
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

#Africa Soil Prediction

##pH is the target variable for predictions. 

####The data have been monotonously transformed from the original measurements and thus include negative values. 

   * PIDN: unique soil sample identifier
   * pH: pH values
   * ELEV: Shuttle Radar Topography Mission elevation data
   * Ref: average long-term Reflectance measurements from MODIS satellite images (Ref2 = red)
   * BSA: average long-term Black Sky Albedo measurements from MODIS satellite images (BSAN = near-infrared)
   * LST: average long-term Land Surface Temperatures from MODIS satellite images (LSTN = night time temperature)
   * Infrared01 - Infrared05: These are 5 bands of infrared absorbance measurements.

In [6]:
data = pd.read_csv("/Users/mrgholt/GADS-22-NYC/Datasets/africa_ph.csv")

#Remove the "Unnamed: 0" column
del data['Unnamed: 0']
del data["PIDN"]

#A brute force approach

In [13]:
predictors = list(data.columns.values)
predictors.remove('pH')
print predictors
print type(predictors)

['ELEV', 'REF2', 'BSAN', 'LSTN', 'Infrared01', 'Infrared02', 'Infrared03', 'Infrared04', 'Infrared05']
<type 'list'>
